<a href="https://colab.research.google.com/github/FreeRikato/Slouch-Detection/blob/master/src/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("hello world")

hello world


In [1]:
print("Colab integration")

Colab integration
